In [2]:
# -*- coding: utf-8 -*-
"""
Transformer Embedding Ensemble with Weighted Soft Voting
Dataset: adjectives_train.csv / adjectives_dev.csv / adjectives_test.csv
Models: RoBERTa, BERT, BART, MiniLM, DistilBERT, DeBERTa
Classifiers: RandomForest, GaussianNB, XGBoost, Linear SVM
Evaluation: 10-fold Cross Validation
"""

# ---------------------------
# Imports
# ---------------------------
import re
import pandas as pd
import numpy as np
import nltk
import torch
from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from transformers import AutoTokenizer, AutoModel

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# ---------------------------
# Step 1: Load Data
# ---------------------------
train_path = r"adjectives_train.csv"
dev_path   = r"adjectives_dev.csv"
test_path  = r"adjectives_test.csv"

print("=== Loading Dataset ===")
train_df = pd.read_csv(train_path)
dev_df   = pd.read_csv(dev_path)
test_df  = pd.read_csv(test_path)

print(f"Train File: {train_path} -> {train_df.shape[0]} samples, {train_df.shape[1]} columns")
print(f"Dev File  : {dev_path} -> {dev_df.shape[0]} samples, {dev_df.shape[1]} columns")
print(f"Test File : {test_path} -> {test_df.shape[0]} samples, {test_df.shape[1]} columns\n")

# Combine train + dev
train_df = pd.concat([train_df, dev_df], ignore_index=True)

X_train_raw = train_df["review"]
y_train = train_df["sentiment_label"]
X_test_raw = test_df["review"]

# ---------------------------
# Step 2: Preprocessing
# ---------------------------
print("Step 2: Preprocessing text...")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    return text

def preprocess_text(text):
    text = clean_text(text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

X_train_processed = X_train_raw.apply(preprocess_text)
X_test_processed = X_test_raw.apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)

num_classes = len(label_encoder.classes_)
print(f"Classes: {label_encoder.classes_}\n")

# ---------------------------
# Step 3: Transformer Embeddings
# ---------------------------
print("Step 3: Generating embeddings...")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transformer_models = {
    "roberta": "roberta-base",
    "bert": "bert-base-uncased",
    "bart": "facebook/bart-base",
    "minilm": "nreimers/MiniLM-L6-H384-uncased",
    "distilbert": "distilbert-base-uncased",
    "deberta": "microsoft/deberta-base"
}

def get_embeddings(texts, model_name, batch_size=16, max_len=128):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    all_embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Embedding {model_name}"):
            batch = texts[i:i+batch_size].tolist()
            encodings = tokenizer(batch, padding=True, truncation=True,
                                  max_length=max_len, return_tensors="pt").to(device)
            outputs = model(**encodings)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            all_embeddings.append(cls_embeddings.cpu().numpy())
    return np.vstack(all_embeddings)

# Training embeddings
train_embeddings = {}
for name, model_name in transformer_models.items():
    train_embeddings[name] = get_embeddings(X_train_processed, model_name)

X_train_emb = np.hstack(list(train_embeddings.values()))

# Test embeddings
test_embeddings = {}
for name, model_name in transformer_models.items():
    test_embeddings[name] = get_embeddings(X_test_processed, model_name)

X_test_emb = np.hstack(list(test_embeddings.values()))

print(f"Training Embedding Shape: {X_train_emb.shape}")
print(f"Test Embedding Shape    : {X_test_emb.shape}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


=== Loading Dataset ===
Train File: adjectives_train.csv -> 6400 samples, 13 columns
Dev File  : adjectives_dev.csv -> 1600 samples, 13 columns
Test File : adjectives_test.csv -> 2000 samples, 13 columns

Step 2: Preprocessing text...
Classes: [0 1]

Step 3: Generating embeddings...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Embedding microsoft/deberta-base: 100%|██████████████████████████████████████████████| 500/500 [22:45<00:00,  2.73s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Embedding microsoft/deberta-base: 100%|██████████████████████████████████████████████| 125/125 [05:35<00:00,  2.68s/it]


Training Embedding Shape: (8000, 4224)
Test Embedding Shape    : (2000, 4224)


In [ ]:
# -*- coding: utf-8 -*-
"""
Transformer Embeddings + SVM-RBF Classifier
10-Fold Cross Validation with Best Hyperparameters
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

# ---------------------------
# Config
# ---------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
FOLDS = 10
RANDOM_STATE = 42

np.random.seed(SEED)
torch.manual_seed(SEED)

# ---------------------------
# Embedding extraction
# ---------------------------
def get_embeddings(model_name, texts, batch_size=16, max_len=128):
    """Extract CLS embeddings from transformer"""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(DEVICE)
    model.eval()

    all_embeddings = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Embedding {model_name}"):
            batch_texts = texts[i:i+batch_size]
            enc = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_len,
                return_tensors="pt"
            ).to(DEVICE)

            outputs = model(**enc)
            # Take [CLS] token (first hidden state)
            embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)

# ---------------------------
# Cross-validation + SVM
# ---------------------------
def run_cv(X, y, model_name):
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)

    # Hyperparameter grid for SVM-RBF
    param_grid = {
        "C": [0.1, 1, 10],
        "gamma": ["scale", "auto", 0.01, 0.001, 0.005],
        "kernel": ["rbf"],
    }

    all_results = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n=== {model_name} | Fold {fold}/{FOLDS} ===")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = np.array(y)[train_idx], np.array(y)[val_idx]

        svm = SVC()
        grid = GridSearchCV(svm, param_grid, scoring="f1_macro", cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_val)

        acc = accuracy_score(y_val, y_pred) * 100
        prec = precision_score(y_val, y_pred, average="macro") * 100
        rec = recall_score(y_val, y_pred, average="macro") * 100
        f1 = f1_score(y_val, y_pred, average="macro") * 100

        print(f"Best Params: {grid.best_params_}")
        print(f"Fold {fold} -> Acc: {acc:.2f} | Prec: {prec:.2f} | Rec: {rec:.2f} | F1: {f1:.2f}")

        all_results.append({
            "fold": fold,
            "acc": acc, "prec": prec, "rec": rec, "f1": f1,
            "best_params": grid.best_params_
        })

    # Aggregate results
    df_results = pd.DataFrame(all_results)
    mean_results = df_results.mean(numeric_only=True).to_dict()

    return mean_results, all_results

# ---------------------------
# Run pipeline
# ---------------------------
# ---------------------------
# Run pipeline
# ---------------------------
final_summary = {}

for model_name, model_ckpt in transformer_models.items():
    print("\n" + "="*20)
    print(f" Running {model_name} ({model_ckpt})")
    print("="*20)

    embeddings = get_embeddings(model_ckpt, texts)
    mean_results, all_folds = run_cv(embeddings, labels, model_name)

    # Save results
    final_summary[model_name] = {
        "accuracy": mean_results["acc"],
        "precision": mean_results["prec"],
        "recall": mean_results["rec"],
        "f1": mean_results["f1"],
    }

    # --- Print final results block ---
    print(f"\n>>> {model_name} Final CV Results ({FOLDS} folds)")
    print(f"Accuracy: {mean_results['acc']:.2f}")
    print(f"Precision: {mean_results['prec']:.2f}")
    print(f"Recall: {mean_results['rec']:.2f}")
    print(f"F1: {mean_results['f1']:.2f}")
    print("="*60)




========== BART Training (10-Fold CV) ==========
Model Spec: 12 layers, 1024 hidden, 16 heads
Best Params: {'C': 1.0, 'gamma': 0.01, 'kernel': 'rbf'}

========== BART | Fold 1/10 ==========
Epoch   1/100 - acc: 61.00% - prec: 69.65% - rec: 70.17% - f1: 69.56%
Epoch   2/100 - acc: 61.34% - prec: 69.87% - rec: 70.18% - f1: 70.20%
Epoch   3/100 - acc: 61.22% - prec: 70.42% - rec: 70.53% - f1: 70.29%
Epoch   4/100 - acc: 61.17% - prec: 70.75% - rec: 70.21% - f1: 70.74%
Epoch   5/100 - acc: 61.08% - prec: 70.81% - rec: 70.34% - f1: 70.66%
Epoch   6/100 - acc: 61.00% - prec: 70.60% - rec: 70.42% - f1: 71.11%
Epoch   7/100 - acc: 61.31% - prec: 71.11% - rec: 71.00% - f1: 71.13%
Epoch   8/100 - acc: 61.27% - prec: 71.21% - rec: 70.61% - f1: 71.07%
Epoch   9/100 - acc: 61.05% - prec: 71.35% - rec: 70.99% - f1: 71.73%
Epoch  10/100 - acc: 61.16% - prec: 71.56% - rec: 71.28% - f1: 71.87%
--- Fold 1 Final ---
Accuracy: 61.16% | Precision: 71.56% | Recall: 71.28% | F1: 71.87%

========== BART | Fo